In [38]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, concatenate
from xgboost import XGBRegressor
from pyswarm import pso

In [39]:
# 1. Data Loading and Preprocessing
def load_and_preprocess_data(file_path):
    # Load data
    data = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    
    # Scale demand
    scaler = MinMaxScaler()
    data['Peak Demand'] = scaler.fit_transform(data[['Peak Demand']])
    
    return data, scaler

In [40]:
# 2. Weather Data Integration
def get_weather_data(api_key, city='Delhi'):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key, 'units': 'metric'}
    response = requests.get(base_url, params=params)
    weather_data = response.json()
    
    if response.status_code == 200:
        temperature = weather_data['main']['temp']
        humidity = weather_data['main']['humidity']
        return temperature, humidity
    else:
        print(f"Error fetching weather data: {weather_data['message']}")
        return None, None

In [41]:
# 3. Feature Engineering
def create_features(data):
    data['hour'] = data.index.hour
    data['day_of_week'] = data.index.dayofweek
    data['month'] = data.index.month
    
    # Fetch weather data (example usage)
    api_key = "91508c6bae1e012fd109256f4422e3fd"
    temperature, humidity = get_weather_data(api_key)
    if temperature is not None and humidity is not None:
        data['temperature'] = temperature
        data['humidity'] = humidity
    else:
        data['temperature'] = np.nan  # Handle missing weather data
        data['humidity'] = np.nan
        
    return data

In [42]:
# 4. Data Preparation for LSTM
def prepare_lstm_data(data, time_steps=30):  # 30-day lookback
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:(i + time_steps)].values)
        y.append(data['Peak Demand'].iloc[i + time_steps])
    return np.array(X), np.array(y)

In [43]:
# 5. Hybrid Model Building Blocks
def build_lstm_attention(input_shape):
    lstm_input = Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(lstm_input)
    attention_out = Attention()([lstm_out, lstm_out])
    lstm_output = LSTM(32)(attention_out)
    return lstm_input, lstm_output

def build_xgboost():
    xgb = XGBRegressor(objective='reg:squarederror', n_estimators=100)
    return xgb

In [44]:
# 6. Hybrid Model Creation
def create_hybrid_model(lstm_input_shape, num_xgb_features):
    # LSTM branch
    lstm_input, lstm_output = build_lstm_attention(lstm_input_shape)
    
    # XGBoost branch
    xgb_input = Input(shape=(num_xgb_features,))
    xgb_output = Dense(16)(xgb_input)  # Simple dense layer for XGB features
    
    # Combine
    combined = concatenate([lstm_output, xgb_output])
    dense1 = Dense(32, activation='relu')(combined)
    output = Dense(1, activation='linear')(dense1)  # Prediction
    
    model = Model(inputs=[lstm_input, xgb_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

In [45]:
# 7. PSO Optimization (Simplified)
def objective_function(params, X_lstm_train, X_xgb_train, y_train, lstm_input_shape, num_xgb_features):
    # Example: params = [lstm_units, learning_rate, xgboost_estimators]
    lstm_units = int(params[0])  # Convert to integer
    learning_rate = params[1]
    xgboost_estimators = int(params[2]) # Convert to integer
    
    # Re-define model with new params
    def create_hybrid_model(lstm_input_shape, num_xgb_features, lstm_units=lstm_units, learning_rate=learning_rate, xgboost_estimators=xgboost_estimators):
        # LSTM branch
        lstm_input = Input(shape=lstm_input_shape)
        lstm_out = LSTM(lstm_units, return_sequences=True)(lstm_input)
        attention_out = Attention()([lstm_out, lstm_out])
        lstm_output = LSTM(32)(attention_out)
        
        # XGBoost branch
        xgb_input = Input(shape=(num_xgb_features,))
        xgb_output = Dense(16)(xgb_input)  # Simple dense layer for XGB features
        
        # Combine
        combined = concatenate([lstm_output, xgb_output])
        dense1 = Dense(32, activation='relu')(combined)
        output = Dense(1, activation='linear')(dense1)  # Prediction
        
        model = Model(inputs=[lstm_input, xgb_input], outputs=output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
        return model

    model = create_hybrid_model(lstm_input_shape, num_xgb_features, lstm_units, learning_rate, xgboost_estimators)
    model.fit([X_lstm_train, X_xgb_train], y_train, epochs=2, verbose=0)  # Train briefly
    loss = model.evaluate([X_lstm_train, X_xgb_train], y_train, verbose=0)  # Evaluate

    return loss  # PSO minimizes the loss

In [46]:
# 8. Main Execution
if __name__ == "__main__":
    import tensorflow as tf

    # Load and preprocess
    file_path = 'daily_demand_data.csv'
    data, demand_scaler = load_and_preprocess_data(file_path)
    
    # Feature creation
    data = create_features(data)
    data = data.dropna() # removing any NAN values that could arise

    # Prepare LSTM data
    time_steps = 30
    X_lstm, y = prepare_lstm_data(data[['Peak Demand', 'hour', 'day_of_week', 'month', 'temperature', 'humidity']], time_steps)

    # Prepare XGBoost data: Use last values from LSTM sequence (adjust features as needed)
    X_xgb = data[['temperature', 'humidity', 'hour', 'day_of_week', 'month']][time_steps:].values

    # Split data
    X_lstm_train, X_lstm_test, X_xgb_train, X_xgb_test, y_train, y_test = train_test_split(
        X_lstm, X_xgb, y, test_size=0.2, shuffle=False
    )

    # PSO optimization
    lb = [32, 0.001, 50]  # Lower bounds [lstm_units, learning_rate, xgboost_estimators]
    ub = [128, 0.1, 200] # Upper bounds [lstm_units, learning_rate, xgboost_estimators]
    
    lstm_input_shape = (X_lstm_train.shape[1], X_lstm_train.shape[2])
    num_xgb_features = X_xgb_train.shape[1]

    # Redefine the objective function with correct arguments
    def pso_objective(params):
        return objective_function(params, X_lstm_train, X_xgb_train, y_train, lstm_input_shape, num_xgb_features)

    best_params, _ = pso(pso_objective, lb, ub, swarmsize=10, maxiter=5)  # Example PSO run

    # Create and train the final model with optimized parameters
    lstm_units = int(best_params[0])
    learning_rate = best_params[1]
    xgboost_estimators = int(best_params[2])

    # Re-define model with new params
    def create_hybrid_model(lstm_input_shape, num_xgb_features, lstm_units=lstm_units, learning_rate=learning_rate, xgboost_estimators=xgboost_estimators):
        # LSTM branch
        lstm_input = Input(shape=lstm_input_shape)
        lstm_out = LSTM(lstm_units, return_sequences=True)(lstm_input)
        attention_out = Attention()([lstm_out, lstm_out])
        lstm_output = LSTM(32)(attention_out)
        
        # XGBoost branch
        xgb_input = Input(shape=(num_xgb_features,))
        xgb_output = Dense(16)(xgb_input)  # Simple dense layer for XGB features
        
        # Combine
        combined = concatenate([lstm_output, xgb_output])
        dense1 = Dense(32, activation='relu')(combined)
        output = Dense(1, activation='linear')(dense1)  # Prediction
        
        model = Model(inputs=[lstm_input, xgb_input], outputs=output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
        return model

    final_model = create_hybrid_model(lstm_input_shape, num_xgb_features, lstm_units, learning_rate, xgboost_estimators)
    final_model.fit([X_lstm_train, X_xgb_train], y_train, epochs=10, batch_size=32)
    # Evaluate the model
    test_loss = final_model.evaluate([X_lstm_test, X_xgb_test], y_test)

    # Print the test loss
    print(f"Test Loss: {test_loss}")

Stopping search: maximum iterations reached --> 5
Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 52.0178
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0821
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0406
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0336
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0298
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0299
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0256
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0246
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0228
Epoch 10/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0225
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0314
Test Loss: 0.04157620668411255


In [47]:
# Make predictions
y_predicted = final_model.predict([X_lstm_test, X_xgb_test])

# Inverse Transform for scaling
y_predicted = demand_scaler.inverse_transform(y_predicted)
y_test = demand_scaler.inverse_transform(y_test.reshape(-1, 1))

print(f"Final Model Test Loss: {test_loss}")
print("Predicted Peak Demand:", y_predicted)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step
Final Model Test Loss: 0.04157620668411255
Predicted Peak Demand: [[206.53368]
 [204.8596 ]
 [203.1855 ]
 [210.73761]
 [209.9857 ]
 [209.23378]
 [208.20775]
 [206.53368]
 [204.8596 ]
 [203.1855 ]
 [210.73761]
 [197.74579]
 [196.99385]
 [196.24194]
 [195.35318]
 [193.67914]
 [192.00502]
 [198.4977 ]
 [197.74579]
 [196.99385]
 [196.24194]
 [195.35318]
 [193.67914]
 [192.00502]
 [198.4977 ]
 [197.74579]
 [196.99385]
 [196.24194]
 [195.35318]
 [193.67914]
 [192.00505]
 [198.49768]
 [197.74579]
 [196.99385]
 [196.24194]
 [195.35318]
 [193.67914]
 [192.00502]
 [198.4977 ]
 [197.74579]
 [196.99385]
 [196.24194]
 [183.25006]
 [182.49815]
 [180.82455]
 [186.25777]
 [185.50584]
 [184.7539 ]
 [184.00198]
 [183.25006]
 [182.49815]
 [180.82455]
 [186.25777]
 [185.50584]
 [184.7539 ]
 [184.00198]
 [183.25006]
 [182.49815]
 [180.82455]
 [186.25777]
 [185.50584]
 [184.7539 ]
 [184.00198]
 [183.25003]
 [182.49815]
 [180.82455]
 [186.25777]
 [185.50584]
 [184.7539